<a href="https://colab.research.google.com/github/ekyuho/2024-1-iot/blob/main/whatsmatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import requests
import json

# 전시장에서는 지정된 Static IP로 변경 필요
url='http://175.197.63.177:8000/local/api/'

def run(t, method, h, j, p):
    #print(url+t, method, p, j)
    if method=='get': r=requests.get(url+t, headers=h, params=p, timeout=10)
    elif method=='post': r=requests.post(url+t, headers=h, json=j, timeout=10)
    elif method=='put': r=requests.put(url+t, headers=h, json=j, timeout=10)
    return r

In [14]:
print('1. 사용가능한 모든 디바이스 찾기')
r=run('devices', 'get', h={}, j={}, p={})
d1=r.json()
#print(f'got {len(d1)} devices','\n')

# 한집의 표준 모델, 즉 어떤 대상 집에는 다음 4가지의 디바이스가 있다고 가정, Query를 통해 유무 확인
#  entity_id: 사실상 instance_id
#  나중에 와츠매터에서 매핑 정보 확정해 주어야 함.
#  location: 물리적 방 구분 목적인데, 하나의 방으로 가정하고, 우리는 사용하지 않음
# 10월10일 스마트라이프 전시에는 다음 네가지 디바이스를 설치함.
house={'main-light':{'entity_id':'light.nano_vdevo'},
       'desk-light':{'entity_id':'switch.otopeulreiseu_kaendeulweomeo'},
       'curtain':{'entity_id':''},
       'presense-sensor':{'entity_id':'binary_sensor.letianpai_presence_sensor_box_occupancy_2'}
      }

def match(d):
    global house
    for x in d1:
        if house[d]['entity_id']==x['entity_id']:
            house[d]['alias']=x['alias']
            return True
    return False

for d in house:
    if match(d): print('matched', house[d])

1. 사용가능한 모든 디바이스 찾기
matched {'entity_id': 'light.nano_vdevo', 'alias': '라이트 나노 램프'}
matched {'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'alias': '캔들워머램프 김동은 프로 사무실'}
matched {'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'alias': 'LetianPai 재실센서'}


In [15]:
print('2. 각 디바이스 상태')
def status(device):
    d=house[device]
    r=run(f"states/{d['entity_id']}", 'get', h={}, j={}, p={})
    try:
        d['state']=r.json()['state']
    except:
        d['state']=''
    return d

for x in house: print(status(x))

2. 각 디바이스 상태
{'entity_id': 'light.nano_vdevo', 'alias': '라이트 나노 램프', 'state': 'off'}
{'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'alias': '캔들워머램프 김동은 프로 사무실', 'state': 'off'}
{'entity_id': '', 'state': ''}
{'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'alias': 'LetianPai 재실센서', 'state': 'off'}


In [16]:
# {'main-light', 'desk-light', 'curtain', 'presense-sensor'}  {'on', 'off'}
def onoff(device, cmd):
  d=house[device]
  domain=f'{d["entity_id"].split(".")[0]}'
  data={"domain": domain, "service" : f"turn_{cmd}"}
  r=run(f"devices/{d['entity_id']}/command", 'post', h={}, j=data, p={})

print('3. 전등/스위치 on')
for x in house: # {'main-light', 'desk-light', 'curtain', 'presense-sensor'}
  onoff(x, 'on')
  print(status(x))

3. 전등/스위치 on
{'entity_id': 'light.nano_vdevo', 'alias': '라이트 나노 램프', 'state': 'on'}
{'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'alias': '캔들워머램프 김동은 프로 사무실', 'state': 'on'}
{'entity_id': '', 'state': ''}
{'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'alias': 'LetianPai 재실센서', 'state': 'off'}


In [17]:
print('4. 전등/스위치 off')
for x in house: # {'main-light', 'desk-light', 'curtain', 'presense-sensor'}
  onoff(x, 'off')
  print(status(x))

4. 전등/스위치 off
{'entity_id': 'light.nano_vdevo', 'alias': '라이트 나노 램프', 'state': 'off'}
{'entity_id': 'switch.otopeulreiseu_kaendeulweomeo', 'alias': '캔들워머램프 김동은 프로 사무실', 'state': 'off'}
{'entity_id': '', 'state': ''}
{'entity_id': 'binary_sensor.letianpai_presence_sensor_box_occupancy_2', 'alias': 'LetianPai 재실센서', 'state': 'off'}


In [ ]:
d1

In [ ]:
house